In [1]:
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
import os
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
llm = ChatGroq(model="Gemma2-9b-It")

In [2]:
from langgraph.graph import StateGraph, START,END
from langchain_core.messages import BaseMessage,HumanMessage,ToolMessage,SystemMessage
from langchain_core.tools import tool
from langgraph.graph import add_messages
from langgraph.prebuilt import ToolNode
from typing import TypedDict,Annotated,Sequence

### 2 Retrievers

### Langchain-retriever

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings 
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
urls = [
"https://python.langchain.com/docs/introduction/",
"https://python.langchain.com/docs/tutorials/llm_chain/",
"https://python.langchain.com/docs/concepts/chat_models/"
]

docs = [WebBaseLoader(url).load() for url in urls]


In [5]:
docs

[[Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='\n\n\n\n\nIntroduction | 🦜️🔗 LangChain\n\n\n\n\n\n\n\n\nSkip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHo

In [6]:
for subList in docs:
    for doc in subList:
        print(doc)

page_content='




Introduction | 🦜️🔗 LangChain








Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use a vectorstore as a retrieverHow to add memory to chatbotsHow to use example selectorsHow to add a semantic layer over graph databaseHow to invoke runnables in parallelHow to stream chat model responsesHow to add default

In [7]:
all_docs = [doc for subList in docs for doc in subList]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
docs_split= text_splitter.split_documents(all_docs)

In [8]:
print(docs_split)

[Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Introduction | 🦜️🔗 LangChain'), Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augme

In [9]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

# Assuming docs_split is a list of langchain Document objects
embedding = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(
    documents=docs_split,
    embedding=embedding,
    persist_directory="./chroma_db"  # optional but useful
)

retriever = vectorstore.as_retriever()


/var/folders/ny/fdxdj3qx4hx3018r63mxg7y40000gn/T/ipykernel_64578/1604761741.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


In [10]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    name="Lanchain_retriever",
    description="Use this to search langchain related content, of prompts and chat models"
)


In [11]:
retriever_tool

Tool(name='Lanchain_retriever', description='Use this to search langchain related content, of prompts and chat models', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x16209fb00>, retriever=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x139272270>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x1626f94e0>, retriever=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x139272270>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}')

### Langgraph-retriever

In [12]:
urls = [
    "https://langchain-ai.github.io/langgraph/concepts/why-langgraph/",
    "https://langchain-ai.github.io/langgraph/tutorials/get-started/2-add-tools/",
    "https://langchain-ai.github.io/langgraph/tutorials/get-started/3-add-memory/"
]

lg_docs = [WebBaseLoader(url).load() for url in urls]
lg_docs

[[Document(metadata={'source': 'https://langchain-ai.github.io/langgraph/concepts/why-langgraph/', 'title': 'Overview', 'description': 'Build reliable, stateful AI systems, without giving up control', 'language': 'en'}, page_content="\n\n\n\n\n\n\n\n\n\n\n\nOverview\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n          Skip to content\n        \n\n\n\n\n\n\n\n            \n            \nOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!\n\n          \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            LangGraph\n          \n\n\n\n            \n              Overview\n            \n          \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            Initializing search\n          \n\n\n\n\n\n\n\n\n\n\n\n\n    GitHub\n  \n\n\n\n\n\n\n\n\n\n\n          \n  \n  \n    \n  \n  Get started\n\n        \n\n\n\n          \n  \n  \n    \n  \n  Guides\n\n        \n\n\n\n          \n  \n  \n    \n  \n  Reference\n\n       

In [13]:
all_docs_lg = [doc for subList in lg_docs for doc in subList]

text_splitter_lg = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunked_docs_lg = text_splitter_lg.split_documents(all_docs_lg)

from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

# Assuming docs_split is a list of langchain Document objects
embedding = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(
    documents=chunked_docs_lg,
    embedding=embedding,
    persist_directory="./chroma_db"  # optional but useful
)

retriever_lg = vectorstore.as_retriever()

In [14]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool_lg = create_retriever_tool(
    retriever_lg,
    name="Langgraph_retriever",
    description="Use this to search langgraph related content, of adding memory, tools to the llm"
)
retriever_tool_lg

Tool(name='Langgraph_retriever', description='Use this to search langgraph related content, of adding memory, tools to the llm', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x16209fb00>, retriever=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x16339f7d0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x1626f94e0>, retriever=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x16339f7d0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_c

In [15]:
tools = [retriever_tool,retriever_tool_lg]

## Building Graph now

In [27]:
class AgentState(StateGraph):
    messages:Annotated[Sequence[BaseMessage],add_messages]

In [28]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="Gemma2-9b-It")

## Agentic RAG build - build an agent and then bind it with tools

In [30]:
def agent(state:AgentState):
    """
        Invokes the agent model to retrieve relevant information based on the context, Given the current state, it will decide which
        tools to be used to retrieve docs or simply end if no relevant docs to be retrieved

        Args : current state ("messages")

        returns: updated state, with messages appended
    """
    messsages = state['messages']

    llm = ChatGroq(model="Gemma2-9b-It")
    llm.bind_tools(tools=tools)

    response = llm.invoke(messsages)

    return {"messages":[response]}


## now we have to decide what to do with the retrieved documents, whether to generate or re-retrieve

In [ ]:
def decider(state:AgentState):

    from pydantic import BaseModel,Field
    class grade(BaseModel):
        binary_score:str = Field("returns yes or no")

    model = ChatGroq(model="Gemma2-9b-It").bind_tools(tools=tools)
    model_grade = model.with_structured_output(grade)
    
